In [32]:
import pandas as pd
import re
import sys
import os
import re

In [33]:
data_oranda= pd.read_csv("../data/oranda_review.csv")
data_oranda.head()

,이름,작성일자,평점,리뷰 내용,만족도
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요


In [34]:
data_oranda.shape

(1500, 5)

# 데이터 필터링

등록된 리뷰내용이 있는 것만 가져오기

In [35]:

filtered_data_oranda = data_oranda[data_oranda["리뷰 내용"] != "등록된 리뷰내용이 없습니다"]
filtered_data_oranda


,이름,작성일자,평점,리뷰 내용,만족도
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요
...,...,...,...,...,...
837,정*운,2024.07.27,5,배송속도도 제품품질도 맛도 좋아요,맛있어요
840,박아름,2023.12.01,5,빠른배송에 너무 맛있어요!,예상보다 맛있어요
841,변*곤,2024.05.27,5,배송도 빠르고 파손없이 잘 수령하였습니다.,맛있어요
842,김*화,2023.12.07,5,배송도 빠르고~너무 맛잇어요~~~,예상보다 맛있어요


In [36]:
df = filtered_data_oranda

불용어 제거 & 특수문자 제거

In [37]:
from konlpy.tag import Kkma #형태소 분석기
from konlpy.tag import Okt

kkma = Kkma()
okt = Okt()

In [38]:
# sys.path.append(os.path.dirname('PyKoSpacing/'))
# from pykospacing import spacing #한글 문장에서 띄어쓰기 보정 
# spacing = Spacing()  # Spacing 객체 초기화
# pip install git+https://github.com/haven-jeon/PyKoSpacing.git

def preprocessing(review, filter=None):
    total_review = ''
    #인풋리뷰
    for idx in range(len(review)):
        r = review[idx]
        #하나의 리뷰에서 문장 단위로 자르기
        #불필요한 정보제거 name의 각 단어를 review에서 제거 
        # 이름이 주어졌을 경우에만 이름 정보 제거
        if filter:
            filter_parts = filter.split(' ')
            for part in filter_parts:
                r = re.sub(part, '', r)
        sentence = r
        #특수문자, 영어 알파벳, 초성/중성/종성(예: "ㄱ", "ㅏ" 등)을 제거.
        sentence = re.sub('\n','',sentence)
        sentence = re.sub('\u200b','',sentence)
        sentence = re.sub('\xa0','',sentence)
        sentence = re.sub('([a-zA-Z])','',sentence)
        sentence = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',sentence)
        sentence = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',sentence)
        #전처리 후 문장이 비어있으면 다음 리뷰로 넘어감 
        if len(sentence) == 0:
            continue
        #동사를 기본형으로 변환
        sentence = okt.pos(sentence, stem = True)
        word = []
        #명사가 아닌 경우 건너띔(list에 추가 안함함)
        for i in sentence:
            if not i[1] == 'Noun':
                continue
            if len(i[0]) == 1:
                continue
            word.append(i[0])
        #단어들을 공백으로 연결하여 하나의 문장으로 변환
        #문장 끝에 마침표 추가
        #전체 리뷰에 추가
        word = ' '.join(word)
        word += '. '
        total_review += word
    return total_review

In [39]:
# 전처리 수행 #21초 정도 걸리는듯듯
df['전처리된_리뷰'] = df.apply(lambda row: preprocessing([row['리뷰 내용']]), axis=1)

C:\Users\zin\AppData\Local\Temp\ipykernel_13732\447569505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['전처리된_리뷰'] = df.apply(lambda row: preprocessing([row['리뷰 내용']]), axis=1)


In [40]:
df.head()

,이름,작성일자,평점,리뷰 내용,만족도,전처리된_리뷰
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요,지난번 아이 생일 선생님 몇개 보고 장바구니 종종 할인 행사 할인 전혀 장바구니 할...
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요,우주 래인 강정 햇습니 아부지 드릴 아부지 우주 래인 강정 옛날 이제 나이 무가 요...
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요,아부지 둘이서 아부지 옛날 장승 모습 는데 나이 가부장 잘못 뚜디리 기억 아부지 아...
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요,우주 용량 소비 기한 구매 이유 물엿 코팅 달달 식감 과자 특유 냄새 추억 우주 쿠...
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요,클래식 간식 최고봉 달달 간식 주문 옛날 문방구 오란 입천장 이번 구매 수제 강정 ...


특정 단어 제거(이벤트, 체험단 등)

In [41]:
# def remove_rows_with_keywords(dataframe, column_name, keywords):
#     """
#     특정 단어가 포함된 문장을 제거하는 함수.

#     Args:
#         dataframe (pd.DataFrame): 데이터프레임
#         column_name (str): 처리할 열 이름
#         keywords (list): 제거할 단어 리스트

#     Returns:
#         pd.DataFrame: 특정 단어가 포함된 행이 제거된 데이터프레임
#     """
#     # 특정 단어가 포함된 행 제거
#     pattern = "|".join(keywords)  # 키워드 리스트를 정규식 패턴으로 변환
#     filtered_df = dataframe[~dataframe[column_name].str.contains(pattern, na=False)]
#     return filtered_df

# # 사용 예시
# keywords = ["이벤트", "체험단"]
# filtered_data_oranda = remove_rows_with_keywords(data_oranda, "리뷰 내용", keywords)

# # 결과 확인
# print(filtered_data_oranda)
# len(filtered_data_oranda)

Case1: 리뷰 별점 구분없이 키워드 뽑기 vs 리뷰 평점별로 키워드 뽑기

Case2: 문장 별로 나누어서 키워드 뽑기 vs 문장 분리 하지 말기
- 일단 문장 분리하면 키워드가 뽑히지 않는 경우도 있음(문장이 너무 짧아서)


Case3: 명사만 이용

https://chaeeunsong.tistory.com/26 참고고


키워드 후보군 DB 만들기 -> 키워드 선정?

# Model1: summarize_with_sentences / KR-WordRank

In [42]:
from krwordrank.sentence import summarize_with_sentences

def df_sumWordSentence(df, review_col, rate_col=None, filter=None, top_k=5, num_keywords=5, num_keysents=10, full_merge=False, rate_merge=False):
    """
    데이터프레임에서 특정 열을 전처리하고 키워드 추출 결과를 데이터프레임에 저장.

    Parameters:
    - df: pd.DataFrame - 입력 데이터프레임
    - review_col: str - 리뷰 내용을 포함하는 열 이름
    - rate_col: str - 평점 열의 이름 (rate_merge=True인 경우 필요)
    - filter: list[str] - 필터링할 단어 목록
    - top_k: int - 각 리뷰별 상위 키워드 개수
    - num_keywords: int - 추출할 키워드 개수
    - num_keysents: int - 추출할 중요 문장 개수
    - full_merge: bool - True일 경우 모든 리뷰를 합쳐서 처리
    - rate_merge: bool - True일 경우 평점별로 리뷰를 합쳐서 처리

    Returns:
    - result_df: pd.DataFrame - 키워드 열이 추가된 데이터프레임
    """

    if num_keywords < top_k:
        num_keywords = top_k
        
    def extract_keywords(texts, stopwords=None):
        # 필터링 처리
        if filter:
            for word in filter:
                texts = texts.replace(word, '')
        try:
            keywords, _ = summarize_with_sentences(
                texts,
                stopwords=stopwords,
                num_keywords=num_keywords,
                num_keysents=num_keysents
            )
            top_keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)[:top_k]
            return [word for word, _ in top_keywords]
        except ValueError:
            return None

    if full_merge:
        # 모든 리뷰를 하나로 합쳐 처리
        merged_reviews = '. '.join(df[review_col].dropna())
        texts = merged_reviews.split('. ')
        keywords = extract_keywords(texts)
        result_df = pd.DataFrame({'merge_type': ['full'], 'keywords': [keywords]})
        return result_df

    if rate_merge:
        # 평점별로 리뷰를 합쳐 처리
        if rate_col is None:
            raise ValueError("rate_merge=True 인 경우, rate_col을 지정해야 합니다.")
        
        rate_keywords = []
        for rate, group in df.groupby(rate_col):
            merged_reviews = '. '.join(group[review_col].dropna())
            texts = merged_reviews.split('. ')
            keywords = extract_keywords(texts)
            rate_keywords.append({'rate': rate, 'keywords': keywords})
        
        result_df = pd.DataFrame(rate_keywords)
        return result_df

    # 기본: 각 리뷰별 처리
    else:
        krwordrank_results = []
        for idx, row in df.iterrows():
            review_content = row[review_col]
            if pd.isna(review_content):
                krwordrank_results.append(None)
                continue
            
            # 필터링 처리
            if filter:
                for word in filter:
                    review_content = review_content.replace(word, '')
            
            # 문장 리스트
            texts = review_content.split('. ')
            keywords = extract_keywords(texts)
            krwordrank_results.append(keywords)

        # 새로운 열 추가
        df['krwordrank'] = krwordrank_results
        return df


In [43]:
df.head()

,이름,작성일자,평점,리뷰 내용,만족도,전처리된_리뷰
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요,지난번 아이 생일 선생님 몇개 보고 장바구니 종종 할인 행사 할인 전혀 장바구니 할...
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요,우주 래인 강정 햇습니 아부지 드릴 아부지 우주 래인 강정 옛날 이제 나이 무가 요...
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요,아부지 둘이서 아부지 옛날 장승 모습 는데 나이 가부장 잘못 뚜디리 기억 아부지 아...
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요,우주 용량 소비 기한 구매 이유 물엿 코팅 달달 식감 과자 특유 냄새 추억 우주 쿠...
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요,클래식 간식 최고봉 달달 간식 주문 옛날 문방구 오란 입천장 이번 구매 수제 강정 ...


In [44]:
# 함수 호출
full_merge_df  = df_sumWordSentence(df, review_col='전처리된_리뷰', top_k=10, full_merge=True)

In [45]:
full_merge_df.head()

,merge_type,keywords
0,full,"[오란, 간식, 주문, 먹기, 구매, 포장, 아이, 강정, 과자, 우주]"


In [46]:
# 2. 평점별로 리뷰를 묶어서 키워드 추출
rate_merge_df = df_sumWordSentence(df, review_col='전처리된_리뷰', rate_col='평점', rate_merge=True)

In [47]:
rate_merge_df

,rate,keywords
0,1,None
1,2,None
2,3,"[기름, 오란, 냄새, 선물]"
3,4,"[주문, 식감, 포장, 과자, 오란]"
4,5,"[오란, 간식, 주문, 구매, 먹기]"


In [48]:
# 3. 각 리뷰별로 처리
individual_df = df_sumWordSentence(df, review_col='전처리된_리뷰', top_k=5)

C:\Users\zin\AppData\Local\Temp\ipykernel_13732\3785192700.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['krwordrank'] = krwordrank_results


In [49]:
individual_df.head()

,이름,작성일자,평점,리뷰 내용,만족도,전처리된_리뷰,krwordrank
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요,지난번 아이 생일 선생님 몇개 보고 장바구니 종종 할인 행사 할인 전혀 장바구니 할...,[구매]
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요,우주 래인 강정 햇습니 아부지 드릴 아부지 우주 래인 강정 옛날 이제 나이 무가 요...,None
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요,아부지 둘이서 아부지 옛날 장승 모습 는데 나이 가부장 잘못 뚜디리 기억 아부지 아...,[아부]
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요,우주 용량 소비 기한 구매 이유 물엿 코팅 달달 식감 과자 특유 냄새 추억 우주 쿠...,"[포장, 식감, 우주]"
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요,클래식 간식 최고봉 달달 간식 주문 옛날 문방구 오란 입천장 이번 구매 수제 강정 ...,"[네덜란, 오란]"


# Model2: KR-WordRank / KR-WordRank

In [55]:
from krwordrank.word import KRWordRank

def df_krwordRank(df, review_col, rate_col=None, filter=None, top_k=5, num_keywords=5, min_count=1, max_length=10, full_merge=False, rate_merge=False):
    """
    데이터프레임에서 특정 열을 전처리하고 키워드 추출 결과를 데이터프레임에 저장.

    Parameters:
    - df: pd.DataFrame - 입력 데이터프레임
    - review_col: str - 리뷰 내용을 포함하는 열 이름
    - rate_col: str - 평점 열의 이름 (rate_merge=True인 경우 필요)
    - filter: list[str] - 필터링할 단어 목록
    - top_k: int - 각 리뷰별 상위 키워드 개수
    - num_keywords: int - 추출할 키워드 개수
    - min_count: int - 단어 최소 등장 빈도
    - max_length: int - 단어 최대 길이
    - full_merge: bool - True일 경우 모든 리뷰를 합쳐서 처리
    - rate_merge: bool - True일 경우 평점별로 리뷰를 합쳐서 처리

    Returns:
    - result_df: pd.DataFrame - 키워드 열이 추가된 데이터프레임
    """
    if num_keywords < top_k:
        num_keywords = top_k
    # KRWordRank 모델 초기화
    krwordrank = KRWordRank(
        min_count=min_count,
        max_length=max_length,
        verbose=False
    )

    def extract_keywords(texts):
        if not texts or len(' '.join(texts)) < 10:  # 텍스트가 비어있거나 너무 짧은 경우
            print("To Short")
            return None
        try:
            result  = krwordrank.extract(
                texts, 
                beta=0.25, 
                max_iter=50, 
                num_keywords=num_keywords
            )
            keywords = result[0]
            top_keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)[:top_k]
            return [word for word, _ in top_keywords]
        
        except Exception as e:
            print(e)
            return None

    if full_merge:
        # 모든 리뷰를 하나로 합쳐 처리
        merged_reviews = '. '.join(df[review_col].dropna())
        texts = merged_reviews.split('. ')
        keywords = extract_keywords(texts)
        result_df = pd.DataFrame({'merge_type': ['full'], 'keywords': [keywords]})
        return result_df

    if rate_merge:
        # 평점별로 리뷰를 합쳐 처리
        if rate_col is None:
            raise ValueError("rate_merge=True 인 경우, rate_col을 지정해야 합니다.")
        
        rate_keywords = []
        for rate, group in df.groupby(rate_col):
            merged_reviews = '. '.join(group[review_col].dropna())
            texts = merged_reviews.split('. ')
            keywords = extract_keywords(texts)
            rate_keywords.append({'rate': rate, 'keywords': keywords})
        
        result_df = pd.DataFrame(rate_keywords)
        return result_df
    
    else:
        # 기본: 각 리뷰별 처리
        krwordrank_results = []
        for idx, row in df.iterrows():
            review_content = row[review_col]
            if pd.isna(review_content):
                krwordrank_results.append(None)
                continue
            
            # 필터링 처리
            if filter:
                for word in filter:
                    review_content = review_content.replace(word, '')
            
            # 문장 리스트
            texts = review_content.split('. ')
            keywords = extract_keywords(texts)
            krwordrank_results.append(keywords)

        # 새로운 열 추가
        df['krwordrank'] = krwordrank_results
        return df


In [56]:
# 1. 전체 리뷰를 합쳐 키워드 추출
full_merge_df = df_krwordRank(df, review_col='전처리된_리뷰', top_k=10, full_merge=True)

In [57]:
full_merge_df

,merge_type,keywords
0,full,"[오란, 간식, 아이, 과자, 주문, 강정, 먹기, 구매, 포장, 진짜]"


In [58]:
# 2. 평점별로 리뷰를 묶어서 키워드 추출
rate_merge_df = df_krwordRank(df, review_col='전처리된_리뷰', rate_col='평점', rate_merge=True)

'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'


In [59]:
rate_merge_df

,rate,keywords
0,1,"[실망, 보고, 오란, 느낌, 쩐내]"
1,2,None
2,3,"[선물, 냄새, 생각, 리뷰, 오란]"
3,4,None
4,5,"[간식, 오란, 아이, 과자, 주문]"


In [60]:
# 3. 각 리뷰별로 처리
individual_df = df_krwordRank(df, review_col='전처리된_리뷰', top_k=3)

'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' 

C:\Users\zin\AppData\Local\Temp\ipykernel_13732\636867675.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['krwordrank'] = krwordrank_results


In [61]:
filtered_df = individual_df[individual_df['krwordrank'].notna()]
filtered_df

,이름,작성일자,평점,리뷰 내용,만족도,전처리된_리뷰,krwordrank
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요,지난번 아이 생일 선생님 몇개 보고 장바구니 종종 할인 행사 할인 전혀 장바구니 할...,"[오란, 구매, 아이]"
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요,아부지 둘이서 아부지 옛날 장승 모습 는데 나이 가부장 잘못 뚜디리 기억 아부지 아...,"[아부지, 생각, 잘못]"
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요,클래식 간식 최고봉 달달 간식 주문 옛날 문방구 오란 입천장 이번 구매 수제 강정 ...,"[네덜란드, 오란, 간식]"
6,김소연,2024.09.22,5,"✅ 구매동기 직장에서 답례품 1개 받았었는데, 너무 맛있어서 순삭 했거든요. 저희 ...",맛있어요,구매 동기 직장 답례 순삭 저희 엄마 킬러 바로 박스 주문 장점 진짜 전통 오란 입...,"[먹기, 오란, 칼로리]"
8,솔직한깡지,2024.07.14,5,후기가 많이 없어서 고민했는데 와정말 이건 사세요오란다가 쿠팡에도 정말 많았는데 그...,맛있어요,후기 고민 이건 오란 다가 쿠팡 정말 후기 후기 보아 전주 대반 걱정 주문 진짜 여...,"[선물, 후기, 무엇]"
...,...,...,...,...,...,...,...
824,조*은,2024.11.22,4,배송비 저렴하고 빠른대신 모든게 겉포장 비닐이라 항상 이런건 다 터져서 오네요.얼마...,맛있어요,배송비 대신 모든 포장 비닐 항상 얼마나 박스 개봉 엉망 진창 지누 하나 주문 쿠팡...,"[배송, 항상, 대신]"
826,김*서,2024.03.01,3,선물주려고 한건데.ㅜㅜ.주말에 필요해서 로켓이면 날짜 맞추겠지 싶어서 주문을 했습니...,보통이에요,선물 려고 주말 로켓 날짜 주문 훼손 박스 라면 요즘 쿠팡 포장 상태 정말 엉망 좀...,"[선물, 로켓, 생각]"
828,하늘의뜻,2023.12.13,5,다른 과자는 안좋아하고 오란다가 최애 과자라 주문했는데너무 맛있네요 달달하고 바삭바...,예상보다 맛있어요,다른 과자 오란 다가 최애 과자 주문 달달 식감 배송 도착 포장 다음 주문 추천.,"[과자, 주문, 도착]"
830,데에헤헿,2023.12.05,5,오란다는 항상 먹을때마다 좋은거 같은데 우주오란다는 특히 더 바삭한 맛이 느껴져서 ...,맛 평가 없음,오란 항상 우주 오란 바삭 금액 배송 파손 도착.,"[오란, 배송, 금액]"


# Model3: KEYBERT 이용

In [24]:
from keybert import KeyBERT
import pandas as pd

# KeyBERT 모델 초기화
kw_model = KeyBERT()

def df_keybert_keywords(df, review_col, rate_col=None, filter=None, top_k=5, full_merge=False, rate_merge=False):
    """
    데이터프레임에서 KeyBERT를 사용해 키워드 추출 결과를 데이터프레임에 저장.

    Parameters:
    - df: pd.DataFrame - 입력 데이터프레임
    - review_col: str - 리뷰 내용을 포함하는 열 이름
    - rate_col: str - 평점 열의 이름 (rate_merge=True인 경우 필요)
    - filter: list[str] - 필터링할 단어 목록
    - top_k: int - 각 리뷰별 상위 키워드 개수
    - full_merge: bool - True일 경우 모든 리뷰를 합쳐서 처리
    - rate_merge: bool - True일 경우 평점별로 리뷰를 합쳐서 처리

    Returns:
    - result_df: pd.DataFrame - 키워드 열이 추가된 데이터프레임
    """
    def extract_keywords(text, top_k):
        keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(2, 4), use_mmr= True, use_maxsum=True, top_n=top_k, diversity=0.3)
        return [keyword for keyword, score in keywords]


    if full_merge:
        # 모든 리뷰를 하나로 합쳐 처리
        merged_reviews = ' '.join(df[review_col].dropna())
        keywords = extract_keywords(merged_reviews, top_k)
        result_df = pd.DataFrame({'merge_type': ['full'], 'keywords': [keywords]})
        return result_df

    if rate_merge:
        # 평점별로 리뷰를 합쳐 처리
        if rate_col is None:
            raise ValueError("rate_merge=True 인 경우, rate_col을 지정해야 합니다.")
        
        rate_keywords = []
        for rate, group in df.groupby(rate_col):
            merged_reviews = ' '.join(group[review_col].dropna())
            keywords = extract_keywords(merged_reviews, top_k)
            rate_keywords.append({'rate': rate, 'keywords': keywords})
        
        result_df = pd.DataFrame(rate_keywords)
        return result_df
    else: 
        # 기본: 각 리뷰별 처리
        krwordrank_results = []
        for idx, row in df.iterrows():
            review_content = row[review_col]
            if pd.isna(review_content):
                krwordrank_results.append(None)
                continue
            
            # 필터링 처리
            if filter:
                for word in filter:
                    review_content = review_content.replace(word, '')
            
            # 키워드 추출
            keywords = extract_keywords(review_content, top_k)
            krwordrank_results.append(keywords)

        # 새로운 열 추가
        df['keybert_keywords'] = krwordrank_results
        return df


In [25]:
# 1. 전체 리뷰를 합쳐 키워드 추출
full_merge_df = df_keybert_keywords(df, review_col='전처리된_리뷰', full_merge=True)

시간이 5분정도 걸림, 너무 오래걸리지 않나..? 싶음 결과는 잘 나오는듯 다만 이상한 키워드도 많이 나옴

In [26]:
full_merge_df

,merge_type,keywords
0,full,"[와이어 치과 선생님 시중, 암모늄 베이킹파우더 아라비아 보관, 발음 묵음 로망스어..."


In [ ]:
# 2. 평점별로 리뷰를 묶어서 키워드 추출
rate_merge_df = df_keybert_keywords(df, review_col='전처리된_리뷰', rate_col='평점', rate_merge=True)

In [145]:
rate_merge_df

,rate,keywords
0,1,None
1,2,None
2,3,None
3,4,None
4,5,None


In [ ]:
# 3. 각 리뷰별로 처리
individual_df = df_keybert_keywords(df, review_col='전처리된_리뷰', top_k=3)

In [146]:
individual_df

,이름,작성일자,평점,리뷰 내용,만족도,전처리된_리뷰,krwordrank
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요,지난번 아이 생일 선생님 몇개 보고 장바구니 종종 할인 행사 할인 전혀 장바구니 할...,None
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요,우주 래인 강정 햇습니 아부지 드릴 아부지 우주 래인 강정 옛날 이제 나이 무가 요...,None
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요,아부지 둘이서 아부지 옛날 장승 모습 는데 나이 가부장 잘못 뚜디리 기억 아부지 아...,None
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요,우주 용량 소비 기한 구매 이유 물엿 코팅 달달 식감 과자 특유 냄새 추억 우주 쿠...,None
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요,클래식 간식 최고봉 달달 간식 주문 옛날 문방구 오란 입천장 이번 구매 수제 강정 ...,None
...,...,...,...,...,...,...,...
837,정*운,2024.07.27,5,배송속도도 제품품질도 맛도 좋아요,맛있어요,배송 속도 제품 품질.,None
840,박아름,2023.12.01,5,빠른배송에 너무 맛있어요!,예상보다 맛있어요,배송.,None
841,변*곤,2024.05.27,5,배송도 빠르고 파손없이 잘 수령하였습니다.,맛있어요,배송 파손 수령.,None
842,김*화,2023.12.07,5,배송도 빠르고~너무 맛잇어요~~~,예상보다 맛있어요,배송.,None
